In [5]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

brand_name_schema = ResponseSchema(name="brand_name",
                             description="This is the name of the brand")

likelihood_of_success_schema = ResponseSchema(name="likelihood_of_success",
                                      description="This is an integer score between 1-10")

reasoning_schema = ResponseSchema(name="reasoning",
                                    description="This is the reasons for the score")

response_schemas = [brand_name_schema, 
                    likelihood_of_success_schema,
                    reasoning_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"brand_name": string  // This is the name of the brand
	"likelihood_of_success": string  // This is an integer score between 1-10
	"reasoning": string  // This is the reasons for the score
}
```


In [6]:
template_string = """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.

{format_instructions}
"""

In [7]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template_string)  
    ],
    input_variables=["brand_description"],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser # here we add the output parser to the Prompt template
)

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
response = chain.predict_and_parse(brand_description="a cool hip new sneaker brand aimed at rich kids")

In [10]:
print(type(response))
response

<class 'dict'>


{'brand_name': 'LuxKicks',
 'likelihood_of_success': '8',
 'reasoning': "The name is catchy and memorable, and it effectively communicates the brand's target audience of rich kids. However, the success of the brand will also depend on the quality and design of the sneakers themselves."}